In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from dotenv import load_dotenv
import datetime

In [11]:
load_dotenv()
jdbc_driver_path = "postgresql-42.7.4.jar"

In [12]:
spark = SparkSession.builder \
    .appName('Solution') \
    .config("spark.driver.extraClassPath", jdbc_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

In [13]:
jdbcHostname = os.getenv("HOST")
jdbcDatabase = os.getenv("DB_NAME")
jdbcUsername = os.getenv("USER")
jdbcPassword = os.getenv("PASSWORD")
jdbcPort = 5432 
jdbcDriver = "org.postgresql.Driver"

connProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": jdbcDriver
}

jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"


In [14]:
work_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM work)", properties=connProperties)
product_size_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM product_size)", properties=connProperties)
subject_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM subject)", properties=connProperties)
artist_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM artist)", properties=connProperties)
canvas_size_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM canvas_size)", properties=connProperties)
image_link_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM image_link)", properties=connProperties)
museum_hours_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM museum_hours)", properties=connProperties)
museum_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM museum)", properties=connProperties)

### 1.  Fetch all the paintings which are not displayed on any museums?

### 2.  Are there museums without any paintings?